<a href="https://colab.research.google.com/github/MiguelAngeloTr/BIGDATA/blob/main/C2/ProyectoFinal/Proyectofinal1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Estudiantes:

Miguel Angel Jimenez Trochez - 2215407
Juan Pablo Castaño  - 2215929
Christian David Cardenaz  - 2215689


In [1]:
!pip install -q --upgrade pyspark
!pip install -q findspark
!pip install kagglehub[pandas-datasets]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.2/317.2 MB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [2]:
!pip install dask==2024.12.1 distributed==2024.12.1 --force-reinstall


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.4/194.4 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.9/134.9 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 403.7/403.7 kB 27.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.0/278.0 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 763.0/763.0 kB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.4/56.4 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.5/442.5 kB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.7/128.7 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import zipfile
import os, pathlib, PIL, shutil, glob
from google.colab import files

import matplotlib.pyplot as plt
from pandas.core.dtypes.api import is_numeric_dtype, is_string_dtype

from pyspark.sql import SparkSession, DataFrame
from pyspark import SparkFiles
import pyspark.sql.functions as F
import pyspark.sql.window as W
import pyspark.sql.types as T

from dask_ml.model_selection import GridSearchCV, RandomizedSearchCV
from dask.distributed import Client

from pyspark.ml.feature import VectorAssembler, MinMaxScaler, StandardScaler, OneHotEncoder, StringIndexer
from pyspark.ml import Pipeline

from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score, f1_score
from mlxtend.plotting import plot_confusion_matrix
from mlxtend.evaluate import confusion_matrix

ModuleNotFoundError: No module named 'dask_ml'

In [ ]:
files.upload()

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d shantanugarg274/sales-dataset
!ls

In [ ]:
zip_ref = zipfile.ZipFile('sales-dataset.zip', 'r') #localizar el nombre del archivo .zip y colocarlo
zip_ref.extractall('Data') #Extracción de archivos descargados en una carpeta llamada 'files', podria ser cualquier nombre
zip_ref.close()

In [ ]:
spark = SparkSession.builder.appName("LogReg PySpark").getOrCreate()
spark

In [ ]:
datos = spark.read.csv('/content/Data/Sales Dataset.csv',inferSchema=True, header=True)
datos.show(5, truncate=False)

In [ ]:
datos.printSchema()

#Informacion Faltante y Columnas innecesarias


In [ ]:
datos_f1 = datos.drop("Order ID")
datos_f1.show(10, truncate=False)

In [ ]:
def faltantes(df):
  falta = df.select([F.sum(F.col(c).isNull().cast('int')).alias(c) for c in df.columns])
  return falta.show()

In [ ]:
faltantes(datos_f1)

# Analisis Exploratorio de Datos (EDA)

In [ ]:
numeric_features = [t[0] for t in datos_f1.dtypes if t[1] == 'int' or t[1] == 'double']
datos_f1.select(numeric_features).describe().toPandas().transpose()

In [ ]:
datos_EDA = datos_f1.toPandas()
num_list = []
cat_list = []

fig, axes = plt.subplots(2, len(datos_EDA.columns), figsize=(300, 10))  # Ajusta el tamaño según sea necesario
fig.subplots_adjust(hspace=0.4, wspace=0.4)

axes = axes.flatten()

# Contador para los gráficos
graph_count = 0

# Iterar sobre las columnas del DataFrame
for column in datos_EDA.columns:
    if is_numeric_dtype(datos_EDA[column]):
        # Histograma en la primera fila
        sns.histplot(datos_EDA[column], kde=True, ax=axes[graph_count])
        axes[graph_count].set_title(f"{column}")

        # Boxplot en la segunda fila
        sns.boxplot(x=datos_EDA[column], ax=axes[graph_count + len(datos_EDA.columns)])
        axes[graph_count + len(datos_EDA.columns)].set_title(f"{column}")
        num_list.append(column)
        graph_count += 1  # Pasar al siguiente gráfico

    elif is_string_dtype(datos_EDA[column]):
        sns.countplot(data=datos_EDA, x=datos_EDA[column], ax=axes[graph_count])
        axes[graph_count].set_title(f"{column}")
        cat_list.append(column)
        axes[graph_count + len(datos_EDA.columns)].axis("off")
        graph_count += 1  # Pasar al siguiente gráfico

# Mostrar la gráfica
plt.show()


In [ ]:
numeric_data = datos.select(numeric_features).toPandas()
sns.pairplot(numeric_data, height=1.4)
plt.show()

In [ ]:
sns.heatmap(numeric_data.corr('spearman'),annot=True)
plt.show()

Es evidente que no hay variables numéricas altamente correlacionadas. Por lo tanto, las mantendremos todas para el modelo. Sin embargo, las columnas x no son realmente útiles, eliminaremos estas x columnas.

#*Modelo 1*


In [ ]:
from pyspark.sql.functions import udf
from pyspark.ml.linalg import VectorUDT
from pyspark.sql.types import ArrayType, DoubleType
import pandas as pd

In [ ]:
datos_f2 = datos_f1.select(
    'Order Date', 'City', 'State',
    'Category', 'Sub-Category', 'PaymentMode',
    'Quantity', 'Amount', 'Profit', 'Year-Month'
)

cols = datos_f2.columns
datos_f2.printSchema()




In [ ]:
#Etapa de preprocesamiento

salida = 'Profit'
caracteristicas = [car for car in cols if car != salida]

cat = [c for c, t in datos_f2.dtypes if t == 'string' and c != salida]
num = [c for c, t in datos_f2.dtypes if t in ['int', 'double'] and c != salida]

print('Cat:', cat, '\nNum:', num)

indexers = [StringIndexer(inputCol=col, outputCol=f"c_{col}") for col in cat]
encoders = [OneHotEncoder(inputCol=f"c_{col}", outputCol=f"o_{col}") for col in cat]

assembler = VectorAssembler(
    inputCols=num + [f"o_{col}" for col in cat],
    outputCol="features"
)

labelIndexer = StringIndexer(inputCol=salida, outputCol='label') if salida in datos_f2.columns else None


In [ ]:
# Pipeline de transformación
stages = indexers + encoders + [assembler]
if labelIndexer:
    stages.append(labelIndexer)

preprocessor = Pipeline(stages=stages).fit(datos_f2)
datos_f3 = preprocessor.transform(datos_f2)



In [ ]:
# Seleccionar solo las columnas necesarias
selectedCols = ['label', 'features']
datos_f4 = datos_f3.select(selectedCols)

#*División de conjuntos de ajuste (entrenamiento) y prueba*


In [ ]:
# Dividir en entrenamiento y prueba
train, test = datos_f4.randomSplit([0.7, 0.3], seed=2024)
print("Training Dataset Count:", train.count())
print("Test Dataset Count:", test.count())

# -------------------------------
# Convertir DenseVector a array
# -------------------------------
vector_to_array_udf = udf(lambda v: v.toArray().tolist(), ArrayType(DoubleType()))

train = train.withColumn("features_array", vector_to_array_udf("features"))
test = test.withColumn("features_array", vector_to_array_udf("features"))

# Pasar a Pandas
train_pd = train.select("features_array", "label").toPandas()
test_pd = test.select("features_array", "label").toPandas()

# Expandir los arrays en columnas individuales
X_train = pd.DataFrame(train_pd["features_array"].tolist())
X_test = pd.DataFrame(test_pd["features_array"].tolist())
y_train = train_pd["label"]
y_test = test_pd["label"]


#SVC


In [ ]:
cls_SVC = SVC()
param_grid = {"C": [0.01, 0.1, 1.0, 3.0, 10.0],
              "kernel": ["rbf", "poly", "sigmoid", "linear"]
              }


In [ ]:
from dask.distributed import Client

gs_SVC = RandomizedSearchCV(cls_SVC, param_grid, n_iter=10, scheduler=client, random_state=42).fit(X_train, y_train)
print("Mejores hiperparámetros:", gs_SVC.best_params_)